# MVGmeinRad Part 2: Store repeated observation in a SQL-Database


In the [first part](https://nbviewer.jupyter.org/github/MrPreacher/MVGmeinRad/blob/master/MVGmeinRad_Part1.ipynb) of this series we looked at rental bike station information for a single point in time. As mentioned there it is even more interesting to look at changes in the utilization of these stations over time.

[This gread blog-post](https://medium.com/@raoshashank/free-cloud-based-data-scraping-using-aws-e111a950e6b5) explains how to use Amazon Web Service (AWS) to execute a Python script that scraps data from the web repeatedly and writes the result into a MySQL-Database. In this case, we use a small small AWS EC2-Instance to execute the Python script below which queries the station-data every five minutes via Linux-Crontab. As before, the script reads the data in JSON-format into a Pandas DataFrame. Afterwards, the data is pushed to a MySQL-Database run on AWS:

```python
"""
Web-Scrapping Script for MVG-MeinRad Station Information:
"""
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine

#Read in url to scrap:
url = pd.read_csv('MVG_URL.txt', header = None).values[0][0]

#Read in AWS-MySQL Connection String:
with open('/home/ubuntu/Scrapper/connection_string.txt') as f:
    connection_string = f.readline()
engine = create_engine(connection_string, echo=False)

#Read JSON-File into Pandas-DataFrame:
df = pd.read_json(url)   
df["Time"] = dt.datetime.now().strftime("%Y.%m.%d %H:%M:%S") #Add Timestamp

#Export File to DataBase:
df.to_sql(name='MVG', con=engine, if_exists = 'append', index=False)
```


Now we can use the `sqlalchemy` library to import the stored information via SQL-queries:

In [1]:
#Load packages:
import os
os.chdir("D:\\MVGmeinRad")
import datetime as dt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Read login information from textfile:
with open('Database_Information.txt') as f:
    for line in f:
        username, password, host, port, database = line.split(';')
#Create connection engine
connect_string = 'mysql+pymysql://{}:{}@{}:{}/{}?charset=utf8mb4'.format(username, password, host, port, database)
engine = create_engine(connect_string)

Having established a connection to the MySQL Server, we can test wether it works proberly via a simple query:

In [3]:
#Test connection via a simple query (total number of observations in table "MVG"):
pd.read_sql('''Select Now() as TimeStamp, Format(count(*),0) as Observations From MVG;''', con=engine)

,TimeStamp,Observations
0,2019-02-09 23:34:47,"3,624,977"


This does seem to work fine. Now we can query the database via SQL as usual. For instance, we can look at the hourly averages of the number of available bikes and free docks for the station next to the central railway station at the 1th of December:

In [4]:
pd.read_sql(''' Select  Concat(Date(Time), " ", Hour(Time), ":00:00") as Time,  
                        name as Station,
                        Round(Avg(bikes_available),2) as Bikes,
                        Round(Avg(docks_available),2) as Docks,
                        Avg(capacity)        as Capacity
                From MVG 
                Where id = 7
                And Date(Time) = "2018-12-01"
                Group by Hour(Time)
                ;''', con=engine)

,Time,Station,Bikes,Docks,Capacity
0,2018-12-01 0:00:00,HBF / Schottstraße 2,16.00,7.00,23.0000
1,2018-12-01 1:00:00,HBF / Schottstraße 2,15.50,7.50,23.0000
2,2018-12-01 2:00:00,HBF / Schottstraße 2,15.42,7.58,23.0000
3,2018-12-01 3:00:00,HBF / Schottstraße 2,15.42,7.58,23.0000
4,2018-12-01 4:00:00,HBF / Schottstraße 2,16.17,6.75,22.9167
5,2018-12-01 5:00:00,HBF / Schottstraße 2,16.08,6.83,22.9167
6,2018-12-01 6:00:00,HBF / Schottstraße 2,16.00,7.00,23.0000
7,2018-12-01 7:00:00,HBF / Schottstraße 2,16.92,6.08,23.0000
8,2018-12-01 8:00:00,HBF / Schottstraße 2,17.83,5.17,23.0000
9,2018-12-01 9:00:00,HBF / Schottstraße 2,18.50,4.50,23.0000


Even though SQL is a powerful tool, I personally prefer to do the data analysis directly in Python. Thus, lets load the entire table into a Pandas DataFrame:

In [5]:
#Read whole table into Pandas DataFrame
df = pd.read_sql('Select * From MVG;', con=engine)
#Keep only observations from true starting time (before that date lapses and inconsitencies exist)
df = df[pd.to_datetime(df["Time"], format="%Y.%m.%d %H:%M:%S") >= "2018.10.27 06:00:00"]
#Show latest observations
df.tail(10)

,address,address_hint,bikes_available,blocked,capacity,docks_available,id,latitude,longitude,name,Time
3625085,Haltestelle Stadtpark / An der Nikolausschanze,Haltestelle Stadtpark / An der Nikolausschanze,7,0,9,2,124,49.993004,8.282681,Stadtpark,2019.02.10 00:35:02
3625086,Kapellenstraße / Ecke Am Sportfeld,Kapellenstraße / Ecke Am Sportfeld,1,0,8,7,54,50.004795,8.201106,Café Raab / Am Gonsenheimer Wald,2019.02.10 00:35:02
3625087,Eingang Schwimmbad / Wallstraße,Eingang Schwimmbad / Wallstraße,1,0,12,11,8,50.000307,8.255863,Taubertsbergbad,2019.02.10 00:35:02
3625088,Obere Kreuzstraße,Obere Kreuzstraße,4,0,18,14,56,50.012979,8.218160,Schwimmbad Mombach,2019.02.10 00:35:02
3625089,Binger Straße,Binger Straße,4,0,12,8,80,50.024891,8.173474,Binger Straße,2019.02.10 00:35:02
3625090,Haltestelle Hochheimer Straße,Haltestelle Hochheimer Straße,0,0,6,6,90,50.006479,8.288806,Hochheimer Straße,2019.02.10 00:35:02
3625091,Otto-Brunfels-Kreisel,Otto-Brunfels-Schneise,7,0,12,5,11,49.986414,8.295064,Jugendherberge,2019.02.10 00:35:02
3625092,Haltestelle Friedrich-von-Pfeiffer-Weg,Haltestelle Friedrich-von-Pfeiffer-Weg,0,0,15,15,71,49.994348,8.237130,Uni / Wittichweg,2019.02.10 00:35:02
3625093,Obere Zahlbacher Straße / Ecke Kilianweg,Obere Zahlbacher Straße / Ecke Kilianweg,2,0,12,10,123,49.990317,8.256717,Universitätsmedizin Süd,2019.02.10 00:35:02
3625094,Innsbrucker Straße,Insbrucker Straße,5,0,6,1,68,50.005935,8.294045,Wiener Platz,2019.02.10 00:35:02


Furthermore, let's save the file in csv-format to disk. These locas files will serve as starting points for the subsequent analyses in the other notebooks:

In [6]:
#Write CSV-Backupfile to disk
df.to_csv("MVG_"+dt.datetime.today().strftime('%Y%m%d')+".csv"
         , index = False, sep=',', encoding='utf-8')